In [17]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText



In [18]:
torch.cuda.is_available()

True

In [19]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")


ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

In [91]:
type(y_train)

pandas.core.frame.DataFrame

In [226]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, x_train, y_train):
        self.x = x_train
        self.y = y_train
    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        # your code here
        return len(self.y)
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        
        # your code here
        return self.x[index], self.y.iloc[[index]].values
        



In [227]:
def collate_fn(data):
    x, y = zip(*data)
    x = torch.Tensor(x)
    y = torch.Tensor(y)
    return x, y

In [228]:
from torch.utils.data import DataLoader

train_dataset = CustomDataset(x_train_lstm, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, collate_fn=collate_fn)

# for x,y in train_loader:
#       print(x.shape, y.shape)

In [291]:
class Model(nn.Module):

    def __init__(
        self,
        num_of_units,
        ner_limit,
        num_embeddings,
        embedding_dim, 
        kernel_sizes, 
        num_filters, 
        num_classes=4, d_prob=0.5):
        super(Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=24, out_channels=104, kernel_size=kernel_sizes[0], stride=1)
        self.conv2 = nn.Conv1d(in_channels=104, out_channels=num_filters*2, kernel_size=kernel_sizes[1], stride=1)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters*3, kernel_size=kernel_sizes[2], stride=1)
        self.rnn = nn.GRU(input_size=50, hidden_size=256, batch_first= False)
        self.fc1 = nn.Linear(in_features=256, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=7)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        # Max pooling over a (2, 2) window
        print(f'input-{x.shape}')
        x = F.relu(self.conv1(x))
        print(f'first conv-{x.shape}')
        x = F.relu(self.conv2(x))
        print(f'second conv-{x.shape}')
        m = nn.MaxPool1d(2)
        x = m(x)
        print(f'max pool-{x.shape}')
        output, _ = self.rnn(x)
        print(f'rnn output-{output.shape}')
        concat = torch.cat([x, output], dim=0)
        print(f'concat-{concat.shape}')
        concat = self.fc1(torch.cat([x, output], 1))
        concat = F.relu(concat)
        concat = self.dropout(concat)
        output = F.sigmoid(self.fc2(concat))

        return output


model = Model(num_of_units=256, ner_limit=64, num_embeddings=24, embedding_dim=104, kernel_sizes=[2, 3, 4], num_filters=32, num_classes=4, d_prob=0.5)
print(model)

Model(
  (conv1): Conv1d(24, 104, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(104, 64, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(104, 96, kernel_size=(4,), stride=(1,))
  (rnn): GRU(50, 256)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [230]:
# class TextCNN(nn.Module):
#     def __init__(
#         self,
#         num_embeddings,
#         embedding_dim, 
#         kernel_sizes, 
#         num_filters, 
#         num_classes=4, d_prob=0.5):
#         super(TextCNN, self).__init__()
#         self.num_embeddings = num_embeddings
#         self.embedding_dim = embedding_dim
#         self.kernel_sizes = kernel_sizes
#         self.num_filters = num_filters
#         self.num_classes = num_classes
#         self.d_prob = d_prob
#         self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
#         self.conv = nn.ModuleList([nn.Conv1d(in_channels=embedding_dim,
#                                              out_channels=num_filters,
#                                              kernel_size=k, stride=1) for k in kernel_sizes])
#         self.dropout = nn.Dropout(d_prob)
#         self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

#     def forward(self, x):
#         x = self.embedding(x.T).transpose(1, 2)
#         x = [F.relu(conv(x)) for conv in self.conv]
#         x = [F.max_pool1d(c, c.size(-1)).squeeze(dim=-1) for c in x]
#         x = torch.cat(x, dim=1)
#         x = self.fc(self.dropout(x))
#         return torch.sigmoid(x).squeeze()


In [231]:
# model = TextCNN(num_embeddings=24,
#                 embedding_dim=104,
#                 kernel_sizes=[2, 3, 4],
#                 num_filters=32,
#                 num_classes=4, 
#                 d_prob=0.5)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
# criterion = nn.BCELoss()

In [232]:
"""
TODO: Specify loss function (CrossEntropyLoss) and assign it to `criterion`.
Spcify optimizer (SGD) and assign it to `optimizer`.
Hint: the learning rate is usually a small number on the scale of 1e-4 ~ 1e-2
"""

criterion = None
optimizer = None

# your code here
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [233]:
# from torch.utils.data import DataLoader

# def load_data(train_data, val_data):
#     train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
#     val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
#     return train_loader, val_loader


# train_loader, val_loader = load_data(x_train_lstm, y_train)
# for batch_idx, samples in enumerate(zip(train_loader, val_loader)):
#       print(batch_idx, samples.shape)

In [234]:
# def get_subvector_data(size, embed_name, data):
#     if embed_name == "concat":
#         vector_size = 200
#     else:
#         vector_size = 100

#     x_data = {}
#     for k, v in data.items():
#         number_of_additional_vector = len(v) - size
#         vector = []
#         for i in v:
#             vector.append(i)
#         if number_of_additional_vector < 0: 
#             number_of_additional_vector = np.abs(number_of_additional_vector)

#             temp = vector[:size]
#             for i in range(0, number_of_additional_vector):
#                 temp.append(np.zeros(vector_size))
#             x_data[k] = np.asarray(temp)
#         else:
#             x_data[k] = np.asarray(vector[:size])

#     return x_data



In [235]:
num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_number = 32
ner_representation_limit = 64
activation_func = "relu"

sequence_model = "GRU"
sequence_hidden_unit = 256

# x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)

In [292]:
# number of epochs to train the model
# make sure your model finish training within 4 minutes on a CPU machine
# You can experiment different numbers for n_epochs, but even 1 epoch should be good enough.
n_epochs = 1

def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim
    """
    :param model: A CNN model
    :param train_dataloader: the DataLoader of the training data
    :param n_epoch: number of epochs to train
    :return:
        model: trained model
    """
    model.train() # prep model for training

    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for x, y in train_dataloader:
            """
            TODO: Within the loop, do the normal training procedures:
                   pass the input through the model
                   pass the output through loss_func to compute the loss (name the variable as *loss*)
                   zero out currently accumulated gradient, use loss.basckward to backprop the gradients, then call optimizer.step
            """
            # your code here
            print(x.shape, y.shape)
            y_hat = None
            loss = None
            y_hat = model(x)
            y_hat = torch.squeeze(y_hat, 1)
            loss = criterion(y_hat,y)
            loss.backward()
            optimizer.step()
            
            curr_epoch_loss.append(loss.cpu().data.numpy())
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
    return model

In [293]:

model = train_model(model, train_loader)

torch.Size([64, 24, 104]) torch.Size([64, 1, 7])
input-torch.Size([64, 24, 104])
first conv-torch.Size([64, 104, 103])
second conv-torch.Size([64, 64, 101])
max pool-torch.Size([64, 64, 50])
rnn output-torch.Size([64, 64, 256])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 50 but got size 256 for tensor number 1 in the list.

In [ ]:
def eval_model(model, dataloader):
    """
    :return:
        Y_pred: prediction of model on the dataloder.
            Should be an 2D numpy float array where the second dimension has length 2.
        Y_test: truth labels. Should be an numpy array of ints
    TODO:
        evaluate the model using on the data in the dataloder.
        Add all the prediction and truth to the corresponding list
        Convert Y_pred and Y_test to numpy arrays (of shape (n_data_points, 2))
    """
    model.eval()
    Y_pred = []
    Y_test = []
    for data, target in dataloader:
        # your code here
        y_hat = model(data)
        y_hat = torch.argmax(y_hat, dim=1)
        y_pred = y_hat.detach().numpy()
        target = target.detach().numpy()
        Y_pred.append(y_pred)
        Y_test.append(target)
    
    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_test = np.concatenate(Y_test, axis=0)

    return Y_pred, Y_test

In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''
from sklearn.metrics import accuracy_score

y_pred, y_true = eval_model(model, val_loader)
acc = accuracy_score(y_true, y_pred)
print(("Validation Accuracy: " + str(acc)))
assert acc > 0.7, "Validation Accuracy below 0.7 for validation data!"
assert len(y_true) == len(y_pred) == 168, "Output size is wrong"
